In [2]:
#Import Libraries:
import os
from time import sleep
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import pickle

In [54]:
def get_infos(house_soup):
    house_info = dict()
    house_info['Location'] = house_soup.find_all("p")[-7].get_text()
    house_info['Price'] = house_soup.find(class_="price").get_text()
    house_info['Area'] = house_soup.find_all(class_="td-value")[4].get_text()
    return house_info

In [55]:
page = 1
house_infos = list()
while True:
    url = requests.get("https://chothuephongtro.me/da-nang.html?page=" + str(page))
    page_soup = bs(url.text, 'html.parser')
    text = page_soup.find_all("p")
    if(text[1].get_text() == "Chưa có tin đăng trong mục này"):
        break
    houses = page_soup.find_all("a", attrs={'style': 'color: #333333;'}, href=True)
    if (houses == []): break
    for house in houses:
        house_page = requests.get(house['href'])
        house_soup = bs(house_page.text, 'html.parser')
        try: 
            house_infos.append(get_infos(house_soup))
        except: continue
    with open('save_page.txt','w') as file:
        file.write(str(page))
        file.close()
    if (page % 10 == 0):
        with open('../Raw_Data/chothuephongtro.pickle','wb') as handle:
            pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)
    page+=1
if (house_infos):
    with open('../Raw_Data/chothuephongtro.pickle','wb') as handle:
        pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [56]:
with open("../Raw_Data/chothuephongtro.pickle","rb") as handle:
    dataset = pickle.load(handle)
dataset = pd.DataFrame(data = dataset)
dataset

,Location,Price,Area
0,"380 Phố Tôn Đức Thắng, Phường Hòa Minh, Liên C...",2 triệu/tháng,35 m2
1,"346 Đường Ngũ Hành Sơn, Phường Mỹ An, Ngũ Hành...",2.5 triệu/tháng,35 m2
2,"582 Đường số 2/9, Phường Hòa Cường Nam, Hải Ch...",2 triệu/tháng,35 m2
3,"559 Trần Cao Vân Đường Trần Cao Vân, Phường Xu...",1.7 triệu/tháng,20 m2
4,"Phường Hòa Cường Bắc, Hải Châu, Đà Nẵng",2.2 triệu/tháng,18 m2
...,...,...,...
373,"31/3 Đường Bắc Đẩu, Hải Châu, Đà Nẵng",2 triệu/tháng,25 m2
374,"Đường Dương Khuê, Ngũ Hành Sơn, Đà Nẵng",2.3 triệu/tháng,25 m2
375,"Đường Trần Cao Vân, Thanh Khê, Đà Nẵng",Thỏa thuận,35 m2
376,"k482 Đường Hoàng Diệu, Hải Châu, Đà Nẵng",1.7 triệu/tháng,28 m2


In [ ]:
dataset.to_csv('../Raw_Data/chothuephongtro.csv')